In [1]:
import nltk
from nltk.corpus import stopwords
import random
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.utils import shuffle
import string
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def find_features(top_1000_words, text):
    feature = {}
    for word in top_1000_words:
        feature[word] = word in text.lower()
    return feature

In [3]:
df = pd.read_csv('gender-classifier-DFE-791531.csv')
df = shuffle(shuffle(shuffle(df)))
df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
11566,815730956,False,finalized,3,10/27/15 0:40,brand,1.0000,yes,1.0,4/1/14 12:11,...,https://pbs.twimg.com/profile_images/583747416...,0,0,@planetgreentree Hi! Thanks for following us! ...,NaN,1761,10/26/15 13:20,6.587400e+17,"Nederland, Colorado",Mountain Time (US & Canada)
296,815719522,False,finalized,3,10/27/15 0:18,female,1.0000,yes,1.0,1/13/15 5:14,...,https://pbs.twimg.com/profile_images/563957664...,0,DFDFDF,‰ÛÏThat‰Ûªs the way you drop em‰Û - #BigTits ...,NaN,3567,10/26/15 12:40,6.587300e+17,"USA, Ashland",Pacific Time (US & Canada)
9700,815729055,False,finalized,3,10/26/15 23:27,brand,1.0000,yes,1.0,8/18/14 6:46,...,https://abs.twimg.com/sticky/default_profile_i...,0,C0DEED,Get Weather Updates from The Weather Channel. ...,NaN,43694,10/26/15 12:39,6.587300e+17,NaN,NaN
6145,815725455,False,finalized,3,10/27/15 1:41,brand,1.0000,yes,1.0,2/6/14 10:41,...,https://abs.twimg.com/sticky/default_profile_i...,0,C0DEED,Get Weather Updates from The Weather Channel. ...,NaN,39428,10/26/15 12:40,6.587300e+17,NaN,NaN
1125,815720360,False,finalized,3,10/27/15 1:33,brand,1.0000,yes,1.0,9/25/15 23:43,...,https://pbs.twimg.com/profile_images/647663761...,0,C0DEED,EVERYDAYDAYILOVEYOU Forevermore\rWeep with the...,NaN,12178,10/26/15 12:40,6.587300e+17,NaN,NaN
9740,815729095,False,finalized,3,10/27/15 0:03,brand,1.0000,yes,1.0,4/16/15 21:53,...,https://abs.twimg.com/sticky/default_profile_i...,0,C0DEED,Get Weather Updates from The Weather Channel. ...,NaN,77349,10/26/15 12:39,6.587300e+17,NaN,NaN
11738,815731133,False,finalized,3,10/27/15 0:56,brand,1.0000,yes,1.0,8/11/15 11:51,...,https://pbs.twimg.com/profile_images/631176464...,0,C0DEED,@mandycampbell01 Mandy Campbell Follow us toda...,NaN,1359,10/26/15 13:20,6.587400e+17,Canada,NaN
15047,815734524,False,finalized,3,10/26/15 22:56,brand,0.6754,yes,1.0,2/13/11 13:29,...,https://pbs.twimg.com/profile_images/651330021...,0,A8C7F7,Didn't get my contract renewed at work and tha...,NaN,769,10/26/15 13:19,6.587400e+17,svÌ_rije,Stockholm
1487,815720724,False,finalized,3,10/26/15 23:42,female,1.0000,yes,1.0,3/23/09 9:15,...,https://pbs.twimg.com/profile_images/109300939...,0,65B0DA,"Amcor Flexibles and ColaLife, a charity in the...",NaN,2401,10/26/15 12:40,6.587300e+17,Los Angeles,Pacific Time (US & Canada)
347,815719574,False,finalized,3,10/27/15 1:34,male,1.0000,yes,1.0,4/10/10 4:03,...,https://pbs.twimg.com/profile_images/448069047...,0,181A1E,"""All of the computer hacking techniques descri...",NaN,133956,10/26/15 12:40,6.587300e+17,"London, England",London


In [4]:
all_descriptions = df['description']
all_tweets = df['text']
all_genders = df['gender']
all_gender_confidence = df['gender:confidence']
description_tweet_gender = []

In [5]:
# Creation of bag of words for the description
bag_of_words = []
c = 0
stop = stopwords.words('english')
for tweet in all_tweets:
    description = all_descriptions[c]
    gender = all_genders[c]
    gender_confidence = all_gender_confidence[c]
    if (str(tweet) == 'nan' and str(description) == 'nan') or str(gender) == 'nan' or str(gender) == 'unknown' or float(gender_confidence) < 0.8:
        c+=1
        continue
    
    if str(tweet) == 'nan':
        tweet = ''
    if str(description) == 'nan':
        description = ''
    
    # removal of punctuations
    for punct in string.punctuation:
        if punct in tweet:
            tweet = tweet.replace(punct, " ")
        if punct in description:
            description = description.replace(punct, " ")
            
    # adding the word to the bag
    for word in tweet.split():
        if word.isalpha() and word.lower() not in stop:
            bag_of_words.append(word.lower())
    for word in description.split():
        if word.isalpha() and word.lower() not in stop:
            bag_of_words.append(word.lower())
    
    description_tweet_gender.append((tweet+" "+description , gender))
    c += 1

print(len(bag_of_words))
print(len(description_tweet_gender))

231741
13817


In [7]:
# get top 2000 words which will act as our features of each sentence
bag_of_words = nltk.FreqDist(bag_of_words)
top_words = []
for word in bag_of_words.most_common(2000):
    top_words.append(word[0])

top_words[:10]

['co',
 'https',
 'weather',
 'get',
 'love',
 'updates',
 'like',
 'channel',
 'http',
 'one']

In [8]:
# creating the feature set, training set and the testing set
feature_set = [(find_features(top_words, text), gender) for (text, gender) in description_tweet_gender]
training_set = feature_set[:int(len(feature_set)*3/4)]
testing_set = feature_set[int(len(feature_set)*3/4):]

print("Length of feature set", len(feature_set))
print("Length of training set", len(training_set))
print("Length of testing set", len(testing_set))

Length of feature set 13817
Length of training set 10362
Length of testing set 3455


In [9]:
# creating a naive bayes classifier
NB_classifier = nltk.NaiveBayesClassifier.train(training_set)
accuracy = nltk.classify.accuracy(NB_classifier, testing_set)*100
print("Naive Bayes Classifier accuracy = " + str(accuracy))
NB_classifier.show_most_informative_features(20)

Naive Bayes Classifier accuracy = 56.46888567293777
Most Informative Features
              continuous = True            brand : female =     56.8 : 1.0
               promoting = True            brand : female =     30.3 : 1.0
                 husband = True             male : brand  =     27.4 : 1.0
                  proves = True            brand : female =     23.6 : 1.0
               empowered = True            brand : female =     23.6 : 1.0
                 outlook = True            brand : female =     22.8 : 1.0
                register = True            brand : female =     22.0 : 1.0
             spiritually = True            brand : male   =     21.8 : 1.0
                feminist = True           female : brand  =     21.2 : 1.0
                 inquiry = True            brand : female =     20.3 : 1.0
               geography = True            brand : female =     18.7 : 1.0
               dedicated = True            brand : male   =     17.2 : 1.0
                  secu

In [ ]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
accuracy = nltk.classify.accuracy(MNB_classifier, testing_set)*100
print("Multinomial Naive Bayes Classifier accuracy =", (accuracy))

In [ ]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
accuracy = nltk.classify.accuracy(LogisticRegression_classifier, testing_set)*100
print("Logistic Regression classifier accuracy =", accuracy)